# Configuration Management via Pydantic (10 points)

Python is a dynamically typed language. This is one of its biggest strengths and one of its weaknesses. In the last few years, there has been concerted effort in the Python community towards adding language features (such as type annotations) that make it easier for static code analyzers to find issues without compromising on the dynamic nature of the language. 

When writing software, it is essential that your objects have data that are correctly validated. This is especially important for machine learning projects. Unlike traditional software, ML models are complex. Often times, it is not easy to predict the output of a ML model when given an input. This becomes even more complex when multiple people (or worse multiple teams) are involved in the development of the ML model. As ML becomes more complex, this is unfortunately inevitable. When a ML model is not working as expected, it is mindbogglingly complex to figure out the root cause. So, it is essential to have a principled programming techniques to reduce errors due to invalid data and configuration.

In this task, we will take some baby steps by using a widely used python library [Pydantic](https://docs.pydantic.dev/latest/). While it is not designed for ML, we will adapt some of its features for some key steps in ML workflow. You can easily extend the ideas in this task so as to use Pydantic for other tasks such as data validation, model monitoring etc. 

## Pydantic Resources

Pydantic is one of the libraries that have taken the Python community by storm due to its thoughtful design and solving an important problem. I would recommend learning it thoroughly and using it in your projects. For this assignment, there are few key ideas that you want to learn.

1. A good starting point is Python type annotations in general. A good tutorial can be found [here](https://realpython.com/python-type-checking/).

2. Pydantic models: [reference](https://docs.pydantic.dev/latest/concepts/models/)

3. Pydantic fields: [reference](https://docs.pydantic.dev/latest/concepts/fields)

4. Unions (especially, discriminated unions): [reference](https://docs.pydantic.dev/latest/concepts/unions/)

5. Validators (especially field and model validators with a focus on `after` validator): [reference](https://docs.pydantic.dev/latest/concepts/validators/)

## Warmup: Validating Python Function Arguments (ungraded)

During development, it is useful to ensure that your code is always with the right type of arguments. If you use type annotations, then the linter will complain if it is called with incorrect types. It is useful to add more teeth to the complaint. A simple way is to throw an exception if the types of arguments are not correct. However, doing this manually is time consuming. Pydantic provides a validation decorator that makes this easy. You can read more about it [here](https://docs.pydantic.dev/latest/concepts/validation_decorator/). It is typically a good idea to have this validator during development and remove it during production. 
 

## Task 1: Adding Annotations to ML Pipeline (10 points)

In this task, you will be making changes to the file `pipeline_configs.py`. You can test using the command

> rye test -- -m t1pydantic

We will start with the `DatasetConfig` class. You will be adding type annotations to the class so that Pydantic will do the testing automatically. Here are the things to do:

1. Ensure that the `file_path` variable points to a valid file name. So if the file does not exist during the object construction, it should throw an error. It should also have a field description "Path to the dataset file". 
2. `train_size` should be a float with a default value of 0.3 and should take a value between 0.1 and 0.5. It should have a field description "Proportion of data to use for training"
3. `validation_size` should be a float with a default value of 0.1 and should take a value between 0.1 and 0.2. It should have a field description "Proportion of data to use for validation"
4. `test_size` should be a float with a default value of 0.1 and should take a value between 0.1 and 0.3. It should have a field description "Proportion of data to use for testing"
5. `production_size` should be a float with a default value of 0.5 and should take a value between 0.1 and 0.5. It should have a field description "Proportion of data to use for Production"
6. Add a `model_validator` in the after mode such that if the value of train_size, validation_size, test_size and production_size does not add up to 1, it should give an error "Fields train_size, validation_size, test_size, production_size should add up to 1."

Next, we will add some annotations to the three classifier classes. These three classes all inherit from MLModelConfig. However, when I create a MLModelConfig, they should automatically get instantiated into the appropriate child class. We will achieve this using Pydantic's discriminated union concept. Concretely, create a variable called `ml_model_type` that has values `decision_tree` or `naive_bayes` or `knn` and use it to discriminate between them. Create a class called `SimpleClassifier` to do this.

**DecisionTreeConfig**:
1. `ml_model_type` should be a literal `decision_tree`
2. Should have a variable called `random_state` that is an integer whose default value is utils.ANSWER_TO_EVERYTHING . 
3. Should have a variable called `criterion` that can only take three values : gini, entropy and log_loss. It should have a description as "Function to measure the quality of a split".
4. Create a variable called `max_depth` that is an integer with a default value of 1 and takes a value that is at least 1. It should have a description of "Maximum depth of the tree".
5. The variable `min_samples_split` should be an integer with a default value of 2 and should be at least 2. It should have a description "Minimum number of samples required to split an internal node"

**NaiveBayesConfig:**

1. `ml_model_type` should be a literal value `naive_bayes`
2. it should have a field called variant that only takes the value "gaussian" and the description must be "Variant of Naive Bayes to use".


**KNNConfig:**

1. `ml_model_type` should be a literal value `knn`
2. n_neighbors is an integer with a default value of 5 and should be greater than or equal to 1. The description should be "Number of neighbors to use".
3. weights should take only two possible values: uniform or distance. The default value should be uniform and must have a description "Weight function used in prediction".
4. p should be an integer with default value of 2 and must be between 1 and 3. The description must be "Power parameter for the Minkowski metric"